In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [6]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="llama-3.3-70b-versatile")

In [7]:
# simple AI Assistance

while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break


goodbye take care yourself


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store={}


In [10]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
config = {"configurable": {"session_id": "firstchat"}}


In [12]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)


In [13]:
model_with_memory.invoke(("Hi! I'm preet"),config=config).content
model_with_memory.invoke(("tell me what is my name?"),config=config).content


'Your name is Preet.'

In [14]:
# from langchain_community.document_loaders import TextLoader, DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
# from langchain import PromptTemplate
# from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
# from langchain_core.output_parsers import StrOutputParser

# ### Reading the txt files from source directory

# loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
# docs = loader.load()

# ### Creating Chunks using RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=50,
#     chunk_overlap=10,
#     length_function=len
# )
# new_docs = text_splitter.split_documents(documents=docs)
# doc_strings = [doc.page_content for doc in new_docs]

# ###  BGE Embddings

# '''from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
# embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
# )
# '''

# ### Creating Retriever using Vector DB

# db = Chroma.from_documents(new_docs, embeddings)
# retriever = db.as_retriever(search_kwargs={"k": 4})

# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
# prompt = PromptTemplate.from_template(template)

# retrieval_chain = (
#     RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
#     | prompt
#     | llm
#     | StrOutputParser()
#     )


# question ="what is llama3? can you highlight 3 important points?"
# print(retrieval_chain.invoke(question))

In [15]:
### Let's Start with Tools and Agents

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [17]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [18]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)


In [19]:
tool.name

'wikipedia'

In [20]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [21]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [22]:
tool.return_direct

False

In [23]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [24]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

In [25]:
from langchain_core.pydantic_v1 import BaseModel, Field

d:\LangGraph_End_To_End\venv\lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [27]:
# tool=WikipediaQueryRun(
#     name="wiki-tool",
#     description="look up things in wikipedia",
#     args_schema=WikiInputs,
#     api_wrapper=api_wrapper,
#     return_direct=True,
# )

In [28]:
### youtube search tool

In [29]:
from langchain_community.tools import YouTubeSearchTool


In [30]:
tool=YouTubeSearchTool()


In [31]:
tool.description


'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [33]:
tool = TavilySearchResults()
tool.invoke({"query": " Fetches all currently ongoing or live cricket matches including international, domestic, and league tournaments. Filters out previews, opinions, and non-authoritative content."})


C:\Users\PREET\AppData\Local\Temp\ipykernel_18632\834566.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults()


[{'title': 'Cricket Schedule - International, domestic and T20 matches - Cricbuzz',
  'url': 'https://www.cricbuzz.com/cricket-schedule/upcoming-series/league',
  'content': 'Schedule of International, T20 League, Indian, Australian and English domestic cricket matches on Cricbuzz.',
  'score': 0.67140645},
 {'title': 'Cricket Fixtures: List of Ongoing, Upcoming and Concluded Series',
  'url': 'https://www.espncricinfo.com/cricket-fixtures',
  'content': 'Find the list of domestic and international current cricket series, upcoming series, and recently concluded tournaments on our comprehensive cricket',
  'score': 0.6326168},
 {'title': 'Cricket Schedule - Check International and Domestic matches on ...',
  'url': 'https://www.fancode.com/cricket/schedule',
  'content': 'Get the latest schedule, fixtures and results of Cricket international and domestic matches on FanCode. Find current and upcoming Cricket match information',
  'score': 0.56575525},
 {'title': 'Cricket Schedule - Upcom

In [34]:
from langchain import hub


In [35]:
from langchain.agents import AgentExecutor,create_openai_functions_agent


In [36]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [37]:
# tavily_tool = TavilySearchResults()

In [38]:
tools = [tool]


In [39]:
agent = create_openai_functions_agent(llm, tools, prompt)


In [40]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [41]:
agent_executor.invoke({"input": "who was the mahatma gandhi"})




> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'who was the mahatma gandhi', 'output': ''}